In [ ]:
! pip install huggingface_hub

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, logging

In [2]:
# This recipe requires a HuggingFace token to access the mistral models from the hub. You will need to create a HuggingFace login
# and generate a token for use at https://huggingface.co/settings/tokens. You will also need to visit the model card at 
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 and accept the terms to use the model. Once you have generated the token,
# save it in an environment variable named HUGGINGFACE_TOKEN and read it in the recipe as shown in the snippet below.
from huggingface_hub import login
hf_token = os.environ.get('HUGGINGFACE_TOKEN')
login(token=hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/saurabh/.cache/huggingface/token
Login successful


In [3]:
logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map="auto", 
                                             load_in_4bit=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [5]:
prompt = [
    {"role": "user", "content": "What is your favourite country?"},
    {"role": "assistant", "content": "Well, I am quite fascinated with Peru."},
    {"role": "user", "content": "What can you tell me about Peru?"}
]

In [6]:
generation_config = GenerationConfig(
    num_beams=4,
    early_stopping=True,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=model.config.eos_token_id,
    max_new_tokens=900,
)

In [ ]:
encoded_prompt = tokenizer.apply_chat_template(prompt, return_tensors="pt")
model_inputs = encoded_prompt.to(device)

In [8]:
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True,
                               generation_config=generation_config)

In [9]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] What is your favourite country? [/INST]Well, I am quite fascinated with Peru.</s>  [INST] What can you tell me about Peru? [/INST] Peru is a country located in South America. It is bordered by Ecuador to the north, Colombia to the northeast, Brazil to the east, Bolivia and Chile to the south, and the Pacific Ocean to the west. Peru is known for its diverse geography, which includes the Andes Mountains, the Amazon Rainforest, and the Pacific coastline.

Peru is also known for its rich history and culture. The country was once home to several pre-Columbian civilizations, including the Inca Empire, which was one of the largest and most advanced civilizations in the world at the time of its collapse in the 16th century.

Today, Peru is a popular tourist destination, known for its stunning natural beauty, ancient ruins, and vibrant culture. Some of the top attractions in Peru include Machu Picchu, the Amazon Rainforest, and the Nazca Lines. The country is also known for its delic

In [10]:
prompt = [
    {"role": "user", "content": "Mary is twice as old as Sarah presently. Sarah is 6 years old.?"},
    {"role": "assistant", "content": "Well, what can I help you with?"},
    {"role": "user", "content": "Can you tell me in a step by step way on how old Mary will be after 5 years?"}
]

In [11]:
encoded_prompt = tokenizer.apply_chat_template(prompt, return_tensors="pt")
model_inputs = encoded_prompt.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True,
                               generation_config=generation_config)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<s> [INST] Mary is twice as old as Sarah presently. Sarah is 6 years old.? [/INST]Well, what can I help you with?</s>  [INST] Can you tell me in a step by step way on how old Mary will be after 5 years? [/INST] Sure! Here's a step-by-step calculation:

1. Sarah is currently 6 years old.
2. Mary is twice as old as Sarah, so Mary is currently 6 x 2 = 12 years old.
3. After 5 years, Mary will be 12 + 5 = 17 years old.

So, Mary will be 17 years old after 5 years.</s>
